In [45]:
###importamos librerias###
import os
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.io as pio

pd.set_option("display.max_columns", 100)
print("Librerías cargadas. Plotly renderer:", pio.renderers.default)

Librerías cargadas. Plotly renderer: colab


In [46]:
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "statsmodels"])

0

In [47]:
###Cargamos el archivo vehicles_us.csv###
CSV_PATH = Path("../vehicles_us.csv")
assert CSV_PATH.exists(), f"No se encontró el archivo: {CSV_PATH.resolve()}"

df = pd.read_csv(
    CSV_PATH,
    low_memory=False,
    )

df_raw = df.copy()

print(f"Cargado: {CSV_PATH.name}")
print(f"Filas x Columnas: {df.shape[0]:,} x {df.shape[1]}")
print("Columnas:", list(df.columns))
display(df.head())


Cargado: vehicles_us.csv
Filas x Columnas: 51,525 x 13
Columnas: ['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'transmission', 'type', 'paint_color', 'is_4wd', 'date_posted', 'days_listed']


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [48]:
###Diagnosticamos resultados###
print("Tipos de datos:")
display(df.dtypes)

print("\n% de nulos por columna (Top 10):")
display((df.isna().mean().sort_values(ascending=False) * 100).head(10))

print("\nDuplicados:", df.duplicated().sum())

Tipos de datos:


,0
price,int64
model_year,float64
model,object
condition,object
cylinders,float64
fuel,object
odometer,float64
transmission,object
type,object
paint_color,object



% de nulos por columna (Top 10):


,0
is_4wd,50.369723
paint_color,17.985444
odometer,15.316836
cylinders,10.208637
model_year,7.023775
condition,0.000000
model,0.000000
price,0.000000
fuel,0.000000
type,0.000000



Duplicados: 0


In [49]:
###Convertimos el formato de la fecha si se requiere###
if "date_posted" in df.columns:
    df["date_posted"] = pd.to_datetime(df["date_posted"], errors="coerce")
    print("'date_posted' convertido a datetime (coerce).")
else:
    print("No existe columna 'date_posted'")

'date_posted' convertido a datetime (coerce).


In [50]:
df_vis = df.copy()
if "price" in df_vis.columns:
    q1, q99 = df_vis["price"].quantile([0.01, 0.99])
    df_vis = df_vis.query("@q1 <= price <= @q99")
print("df_vis shape:", df_vis.shape)

df_vis shape: (51010, 13)


In [44]:
# $$ Histograma de precios $$
if {"price"} <= set(df_vis.columns):
    px.histogram(df_vis, x="price", nbins=60, title="Distribución de precios").show()

if {"odometer","price"} <= set(df_vis.columns):
    color_col = "model_year" if "model_year" in df_vis.columns else None
    px.scatter(df_vis.dropna(subset=["odometer","price"]),
               x="odometer", y="price", color=color_col,
               trendline="lowess", title="Precio vs Odómetro").show()

if {"condition","price"} <= set(df_vis.columns):
    px.box(df_vis, x="condition", y="price", title="Precio por condición").show()

if {"type"} <= set(df_vis.columns):
    counts = df_vis["type"].value_counts().reset_index()
    counts.columns = ["type","count"]
    px.bar(counts, x="type", y="count", title="Conteo por tipo").update_layout(xaxis_tickangle=-45).show()

if {"posted_month"} <= set(df_vis.columns):
    ts = df_vis.groupby("posted_month").size().rename("count").reset_index()
    px.line(ts, x="posted_month", y="count", title="Publicaciones por mes").show()

if {"age","price"} <= set(df_vis.columns):
    px.scatter(df_vis.dropna(subset=["age","price"]),
               x="age", y="price", trendline="lowess",
               title="Precio vs Edad del vehículo").show()